In [18]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [19]:
data = pd.read_csv('news_sample')
print(data)
data.info()

     Unnamed: 0     id                domain        type  \
0             0    141               awm.com  unreliable   
1             1    256     beforeitsnews.com        fake   
2             2    700           cnnnext.com  unreliable   
3             3    768               awm.com  unreliable   
4             4    791  bipartisanreport.com   clickbait   
..          ...    ...                   ...         ...   
245         245  39259     beforeitsnews.com        fake   
246         246  39468     beforeitsnews.com        fake   
247         247  39477       www.newsmax.com         NaN   
248         248  39550       www.newsmax.com         NaN   
249         249  39558       www.newsmax.com         NaN   

                                                   url  \
0    http://awm.com/church-congregation-brings-gift...   
1    http://beforeitsnews.com/awakening-start-here/...   
2    http://www.cnnnext.com/video/18526/never-hike-...   
3    http://awm.com/elusive-alien-of-the-sea-ca

In [20]:
def clean_data(text):
    # makes the text a string
    text = str(text) 
    # using regular expression to find dates, urls starting with https, urls starting with /, numbers, emails, line breaks or spaces and special characters and then replacing 
    # them with DATE, URL, NUM, EMAIL, a single space and nothing.
    text = re.sub(r'([a-zA-Z]*.? \d{1,2},? \d{4}|[a-zA-Z]{3}.? \d{4}|\d{1,2}[/-]\d{1,2}[/-]\d{4}|\d{4}[/-]\d{1,2}[/-]\d{1,2})','DATE',text) 
    text = re.sub(r'https?://[w]{0,3}\.?[a-z]+\.[a-z]\w*.*|[w]{3}\.\w+\.\w+\w*.*', 'URL', text)
    text = re.sub(r'([w]{3}\.\w+.\w{3})|https?://', 'URL', text)
    text = re.sub(r'(?:^|(?<=[^\w,.]))[+--]?(([1-9]\d{0,2}(,\d{3})+(\.\d*)?)|([1-9]\d{0,2}([ .]\d{3})+(,\d*)?)|(\d*?[.,]\d+)|\d+)(?:$|(?=\b))', 'NUM', text)
    text = re.sub(r'(?:^|(?<=[^\w@.)]))([\w+-](\.(?!\.))?)*?[\w+-]@(?:\w-?)*?\w+(\.([a-z]{2,})){1,3}(?:$|(?=\b))', 'EMAIL', text)
    text = re.sub(r'(\\n)+|(\\t)+|\s{2,}', ' ', text)
    text = re.sub(r'(\.)*(\,)*(\\)*(/)*(#)*(!)*(\$)*(%)*(\^)*(&)*(\*)*(;)*(:)*({)*(})*(=)*(\–)*(_)*(´)*(\')*(~)*(\()*(\))*(\|)*(\“)*(\?)*(\”)*(\-)*(\@)*(\[)*(\])*', '', text)
    # turns all the text to lowercase letters
    text = text.lower()
    # returns the text
    return text

In [21]:
cleaned_data = []
for i in data["content"]:
    cleaned_data.append(clean_data(i))
string_of_contents = " , ".join([content for content in cleaned_data])

In [22]:
stemmer = PorterStemmer()
stopwords = (stopwords.words('english'))
total_cleaned_data = []

for x in cleaned_data:
    tokens = nltk.word_tokenize(x)
    tokens_without_sw = [word for word in tokens if not word in stopwords]
    stemmed_tokens = [stemmer.stem(token) for token in tokens_without_sw]
    joined = " ".join([content for content in stemmed_tokens])
    total_cleaned_data.append(joined)

In [23]:
to_drop = ['content','keywords','summary']
data.drop(to_drop, inplace=True, axis=1)

In [24]:
data['content'] = total_cleaned_data

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        250 non-null    int64 
 1   id                250 non-null    int64 
 2   domain            250 non-null    object
 3   type              238 non-null    object
 4   url               250 non-null    object
 5   scraped_at        250 non-null    object
 6   inserted_at       250 non-null    object
 7   updated_at        250 non-null    object
 8   title             250 non-null    object
 9   authors           170 non-null    object
 10  meta_keywords     250 non-null    object
 11  meta_description  54 non-null     object
 12  tags              27 non-null     object
 13  content           250 non-null    object
dtypes: int64(2), object(12)
memory usage: 27.5+ KB


We would like to make a table in our database where we can find the id for a meta_keyword and also search for a meta_keyword and then we get every id for that word.
We also want to implement a table for types were the different types are not mentioned more times and we can select every type with for example "fake news" and get the id for every article that fits the criteria. 
We want to create small tables with some primary keys that feeds the big table articles by obtaining information from foreign keys from the small tables. 

In [26]:
#make database

try:
        #connect to database
        con = psycopg2.connect("user=postgres password='Loufi2412'")
        con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
                
        #make a cursor
        cur = con.cursor()
        
        #make DB
        
        Createartikledata = 'create database "20"'
        
        cur.execute(Createartikledata)
        con.commit()
    
except:
        ()

#add table to DB
try:
    #connects to DB
    con = psycopg2.connect("user=postgres dbname='20' password='Loufi2412'")
    #make cursor
    cur = con.cursor()
    #creates table
    cur.execute('''CREATE TABLE artikler (
        id text PRIMARY KEY,
        URL text,
        Title text,
        Content text,
        Domain text,
        Type text,
        Authors text,
        Meta_keywords text,
        Meta_description text,
        Tags text,
        scraped_at text,
        inserted_at text,
        updated_at text
    )
    ''')
    con.commit()
except:
    ()
#filters csv file
Artikel = pd.DataFrame(data,columns=['id','url','title','content','domain', 'type', 'authors', 'meta_keywords', 'meta_description',
                                    'tags', 'scraped_at', 'inserted_at', 'updated_at'])

#connects to DB
con = psycopg2.connect("user=postgres dbname='20' password='Loufi2412'")
#make cursor
cur = con.cursor()
    
#inserts into table
i=0
while i < 250:
    row = Artikel.iloc[i,:]
    try:
        cur.execute(
            "INSERT INTO artikler VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
            row
        )
    except:
        ()
    i= i+1
con.commit()

In [27]:
con = psycopg2.connect("user=postgres dbname='20' password='Loufi2412'")
#make cursor
cur = con.cursor()

cur.execute('''
                            SELECT DISTINCT domain FROM artikler
                            WHERE type = 'reliable'
                ''')
reliable = cur.fetchall()
con.commit()
print(reliable)

[]


'Det næste burde virke men den kan ikke finde ud af at matche på '['']'!!!!!!!'

In [28]:
con = psycopg2.connect("user=postgres dbname='20' password='Loufi2412'")
#make cursor
cur = con.cursor()
try:

    cur.execute('''
                    CREATE VIEW meta_sample1 AS 
                    SELECT id,meta_keywords 
                    FROM artikler WHERE meta_keywords <> '[\'\']'
                ''')
except:
    print('fail')

try:
    
    cur.execute('''
                            CREATE VIEW meta_sample2 AS
                            SELECT id,meta_keywords FROM artikler
                            WHERE meta_keywords <> '['']'
                ''')
except:
    print('fail')
con.commit()
con = psycopg2.connect("user=postgres dbname='20' password='Loufi2412'")
#make cursor
cur = con.cursor()
cur.execute('''
                SELECT meta_sample1.id, meta_sample2.id
                FROM meta_sample1
                INNER JOIN meta_sample2 ON meta_sample1.meta_keywords=meta_sample2.meta_keywords AND meta_sample1.id <> meta_sample2.id
                ''')

join_meta=cur.fetchall()
con.commit()
print(join_meta)

[]


In [29]:
con = psycopg2.connect("user=postgres dbname='20' password='Loufi2412'")
#make cursor
cur = con.cursor()
cur.execute ('''
             SELECT COUNT(authors), domain
             FROM artikler
             GROUP BY domain
                    ''')
dom_au=cur.fetchall()
con.commit()
print(dom_au)

[]


In [30]:
con = psycopg2.connect("user=postgres dbname='20' password='Loufi2412'")
#make cursor
cur = con.cursor()
cur.execute ('''
             SELECT COUNT(type='fake'), domain
             FROM artikler
             GROUP BY domain
                    ''')
dom_au=cur.fetchall()
con.commit()
print(dom_au)

[]


In [31]:
con = psycopg2.connect("user=postgres dbname='20' password='Loufi2412'")
#make cursor
cur = con.cursor()
cur.execute ('''
             SELECT COUNT(type='fake'), Authors
             FROM artikler
             GROUP BY Authors
                    ''')
dom_au=cur.fetchall()
con.commit()
print(dom_au)

[]
